In [1]:
import pandas as pd

In [3]:
df_final = pd.read_csv('data_final_sblm_stem.csv')
df_final.head()

,Unnamed: 0,full_text,jenis_ekspedisi,media_scraping,Cleaned_full_text,Cleaned_stopwords_full_text
0,0,\n\n\n\n\n\n\nIkuti kami di Google Berita \nAs...,anteraja,mediakonsumen.com,ikuti kami di google berita assalamualaikum pe...,ikuti google berita assalamualaikum perkenalka...
1,1,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya mau sharing b...,ikuti google berita sharing buruknya tokopedia...
2,2,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya seorang pelan...,ikuti google berita pelanggan setia tokopedia ...
3,3,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya adalah penjua...,ikuti google berita penjual aktif tokopedia to...
4,4,\n\n\n\n\n\n\nIkuti kami di Google Berita \nDe...,anteraja,mediakonsumen.com,ikuti kami di google berita dengan hormat sehu...,ikuti google berita hormat sehubungan surat ir...


In [8]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4352 entries, 0 to 4351
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unnamed: 0                   4352 non-null   int64 
 1   full_text                    4352 non-null   object
 2   jenis_ekspedisi              4352 non-null   object
 3   media_scraping               4352 non-null   object
 4   Cleaned_full_text            4352 non-null   object
 5   Cleaned_stopwords_full_text  4352 non-null   object
dtypes: int64(1), object(5)
memory usage: 204.1+ KB


In [6]:
df_final.jenis_ekspedisi.value_counts()

jnt                 1266
sicepat             1227
jne                 1035
anteraja             550
shopee-express       115
ninja Xpress          89
pt-pos-indonesia      43
tiki                  27
Name: jenis_ekspedisi, dtype: int64

In [7]:
df_final.media_scraping.value_counts()

twitter              2319
mediakonsumen.com    2033
Name: media_scraping, dtype: int64

# next step

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
import emoji
import re
import html

pretrained_id = "mdhugol/indonesia-bert-sentiment-classification"
label_id = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}

In [2]:
md = pd.read_csv('/kaggle/input/dataset-final-inteligo/data_mediakonsumen.csv')
tw = pd.read_csv('/kaggle/input/dataset-final-inteligo/data_twitter.csv')

In [3]:
md = md[['Content','Ekspedisi','media_scraping']]
tw = tw[['full_text','jenis_ekspedisi']]
tw['media_scraping'] = 'twitter'
md.rename(columns={'Content':'full_text','Ekspedisi':'jenis_ekspedisi'},inplace=True)

In [4]:
df_final = pd.concat([md,tw])
df_final

,full_text,jenis_ekspedisi,media_scraping
0,\n\n\n\n\n\n\nIkuti kami di Google Berita \nAs...,anteraja,mediakonsumen.com
1,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com
2,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com
3,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com
4,\n\n\n\n\n\n\nIkuti kami di Google Berita \nDe...,anteraja,mediakonsumen.com
...,...,...,...
2314,"Gak ngerti lagi sama @anteraja_id , same day k...",anteraja,twitter
2315,WTS : 🔴Anteraja edition (R) PC Shani PC Gracia...,anteraja,twitter
2316,@TokopediaCare Ga ada artinya masukan diterima...,anteraja,twitter
2317,Saya order barang online dg layanan kurir same...,anteraja,twitter


In [5]:
pip install sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 8.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import string

def preprocess_tweet(tweet):
    tweet = tweet.lower() #Merubah format teks menjadi format huruf kecil semua (lowercase).
    # Remove HTML tags
    tweet = re.sub(r'<.*?>', '', tweet)
    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    # Remove www URLs
    tweet = re.sub(r'www.\S+', '', tweet)
    # Menghapus username yang diawali dengan "@"
    tweet = re.sub(r'@[\w_]+', '', tweet)
    # Menghapus retweet yang diawali dengan "RT" dan diikuti oleh "@"
    tweet = re.sub(r'RT @[\w_]+:', '', tweet)
    
    #Menghapus karakter angka.
    tweet = re.sub('[^A-Za-z]+', ' ', str(tweet)) 
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = tweet.strip()
    return tweet
df_final['Cleaned_full_text'] = df_final['full_text'].apply(preprocess_tweet)
df_final.head()

,full_text,jenis_ekspedisi,media_scraping,Cleaned_full_text
0,\n\n\n\n\n\n\nIkuti kami di Google Berita \nAs...,anteraja,mediakonsumen.com,ikuti kami di google berita assalamualaikum pe...
1,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya mau sharing b...
2,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya seorang pelan...
3,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya adalah penjua...
4,\n\n\n\n\n\n\nIkuti kami di Google Berita \nDe...,anteraja,mediakonsumen.com,ikuti kami di google berita dengan hormat sehu...


In [7]:
from nltk.corpus import stopwords
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# get Indonesian stopword 
list_stopwords = set(stopwords.words('indonesian'))
def remove_short_words(tweet):
         # split the text into words
        words = tweet.split()
        # filter out words with less than 2 characters
        words = [word for word in words if len(word) > 2]
        tokens_without_stopword = [word for word in words if not word in list_stopwords]
        # join the remaining words into a string
        tweet = ' '.join(tokens_without_stopword)
        return tweet
df_final['Cleaned_stopwords_full_text'] = df_final['Cleaned_full_text'].apply(remove_short_words)
df_final.head()

,full_text,jenis_ekspedisi,media_scraping,Cleaned_full_text,Cleaned_stopwords_full_text
0,\n\n\n\n\n\n\nIkuti kami di Google Berita \nAs...,anteraja,mediakonsumen.com,ikuti kami di google berita assalamualaikum pe...,ikuti google berita assalamualaikum perkenalka...
1,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya mau sharing b...,ikuti google berita sharing buruknya tokopedia...
2,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya seorang pelan...,ikuti google berita pelanggan setia tokopedia ...
3,\n\n\n\n\n\n\nIkuti kami di Google Berita \nSa...,anteraja,mediakonsumen.com,ikuti kami di google berita saya adalah penjua...,ikuti google berita penjual aktif tokopedia to...
4,\n\n\n\n\n\n\nIkuti kami di Google Berita \nDe...,anteraja,mediakonsumen.com,ikuti kami di google berita dengan hormat sehu...,ikuti google berita hormat sehubungan surat ir...


In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
def stemming(tweet):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    tweet = stemmer.stem(tweet)
    return tweet
df_final['Cleaned_stopwords_stemming_full_text'] = df_final['Cleaned_stopwords_full_text'].apply(stemming)
df_final.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_id)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_id)
sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [ ]:
bacaan = []
label = []
nilai = []
for i in range(0,len(df_final['Cleaned Comment'])):
    #text = preprocess_tweet()
    text = df_final['Cleaned Comment'].iloc[i]
    result = sentiment_analysis(text)
    status = label_id[result[0]['label']]
    score = result[0]['score']
    bacaan.append(text)
    label.append(status)
    nilai.append(score)
df_hasil = pd.DataFrame({'Text':bacaan,'status':label,'score':nilai})
df_hasil